# Measure the adoption of your Amazon QuickSight dashboards and manage the assets [2025 version]

In [12]:
from IPython.display import HTML

HTML('<a href="https://d172fagwhsdxeg.cloudfront.net/index.html?dashboardid=deeb0ed5-3260-4023-8b11-dcd604d76332&dashboardregion=us-east-1" target="_blank">Click here to open the sample dashboard</a>')


#show_webpage("https://d172fagwhsdxeg.cloudfront.net/index.html?dashboardid=deeb0ed5-3260-4023-8b11-dcd604d76332&dashboardregion=us-east-1")

In [24]:
from IPython.display import HTML


HTML('<a href="https://aws.amazon.com/blogs/big-data/using-administrative-dashboards-for-a-centralized-view-of-amazon-quicksight-objects/" target="_blank">Click here to open the 2020 blog</a>')



In [25]:
from IPython.display import HTML


HTML('<a href="https://aws.amazon.com/blogs/big-data/building-an-administrative-console-in-amazon-quicksight-to-analyze-usage-metrics/" target="_blank">Click here to open the 2021 blog</a>')




In [26]:
from IPython.display import HTML

HTML('<a href="https://aws.amazon.com/blogs/business-intelligence/measure-the-adoption-of-your-amazon-quicksight-dashboards-and-view-your-bi-portfolio-in-a-single-pane-of-glass/" target="_blank">Click here to open the 2022 blog</a>')




## The sample screenshot

In [15]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://d2908q01vomqb2.cloudfront.net/b6692ea5df920cad691c20319a6fffd7a4a766b8/2022/10/13/l4-1024x610.png")

## The architecture diagram

In [27]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://d32le6a78ncrct.cloudfront.net/image/Admin_Console_Arch.drawio.png")

**Above workflow involves the following steps:**

1. The AWS Glue Python Shell job is scheduled to run hourly on regular basis. This process calls QuickSight APIs to get the QuickSight namespace, group, user, and asset access permissions information and save the results into Amazon Simple Storage Service (Amazon S3) bucket.
2. AWS CloudTrail logs are stored in an S3 bucket.
3. Based on the file in Amazon S3 that contains user-group information, dataset information, QuickSight assets access permissions information, as well as dashboard views and user login events from the CloudTrail logs, five Amazon Athena tables are created. Optionally,
    a. You can access the Amazon S3 data using Amazon Redshift spectrum tables
    b. Also, the BI engineer can combine these tables with employee information tables to display human resource information of the users.
4. Four QuickSight datasets fetch the data from the Athena tables created in Step 3 and import them into SPICE. Then, based on these datasets, a QuickSight dashboard is created.

**Ref**
https://docs.aws.amazon.com/quicksight/latest/user/logging-non-api.html

**Prerequisites**
For this walkthrough, you should have the following prerequisites:

* An AWS account
* Access to the following AWS services:
    * AWS Athena
    * AWS CloudFormation
    * AWS QuickSight
    * Amazon Simple Storage Service (Amazon S3)
    * AWS Glue
    * Amazon Redshift (Optional)
    * Basic knowledge of Python
    * Basic knowledge of SQL

### CloudFormation template for AWS Glue Python Shell job

In [3]:
from IPython.display import HTML

image_url = "https://d32le6a78ncrct.cloudfront.net/image/Picture1.png"
link_url = "https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/template?stackName=adminconsoleglue&templateURL=https://admin-console-cfn-dataprepare-code.s3.us-east-1.amazonaws.com/cfn/admin_console_glue_2025_folder_q-updated.template"

HTML(f'<a href="{link_url}" target="_blank"><img src="{image_url}" width="300"/></a>')

* In the Glue console, manually run the `adminconsoledatasetdashboardinfo` and `adminconsoleuserdataaccessinfo` jobs to quickly generate the datasets instead of waiting for the next scheduled run.
* Verify the data is available in the `admin-console-new-[AWS-account-ID]` S3 bucket after the jobs complete.
* If data is in the S3 bucket, continue deployment of the remaining modules.

### Enable CloudTrail Log

* Create a CloudTrail log if you don’t already have one and note down the S3 bucket name of the log files for future use.
* Note down all the resources created from the previous steps. If the S3 bucket name for the CloudTrail log from step 2 is different from the one in step 1’s output, use the S3 bucket from step 2.

### CloudFormation template for Athena tables<br>
To create your Athena tables, complete the following steps:<br>
The following table summarizes the keys and values you use when creating the Athena tables with the next CloudFormation stack.

| Key            | Value                       | Description                                                           |
|----------------|-----------------------------|-----------------------------------------------------------------------|
| CloudtrailLocation         | for example: cloudtrail-awslogs-[aws-account-id]-do-not-delete/AWSLogs/[aws-account-id]           | The Amazon S3 location of the CloudTrail log end with `CloudTrail/`                      |
| StartDateParameter        | for example: `2023/02/01`       | The date for the first log CloudTrail delivered to that location, in YYYY/MM/DD format         |


You can find the location of your CloudTrail logs in the CloudTrail management console:

In [22]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://d2908q01vomqb2.cloudfront.net/d02560dd9d7db4467627745bd6701e809ffca6e3/2024/09/19/CT1.png")

And the date of the first log file delivered by CloudTrail in the S3 management console:

In [25]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://d2908q01vomqb2.cloudfront.net/d02560dd9d7db4467627745bd6701e809ffca6e3/2024/09/19/CT2.png")

### Choose the below Launch Stack to create the Athena tables.

In [2]:
from IPython.display import HTML

image_url = "https://d32le6a78ncrct.cloudfront.net/image/Picture1.png"
link_url = "https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/template?stackName=adminconsoleathena&templateURL=https://admin-console-cfn-dataprepare-code.s3.us-east-1.amazonaws.com/cfn/admin-console-athena-tables-2025-folder_q-updated.template"

HTML(f'<a href="{link_url}" target="_blank"><img src="{image_url}" width="300"/></a>')

After a successful deployment, you have a database called `admin-console-2025` created in AwsDataCatalog in Athena and five tables in the database: `cloudtrail_logs_pp`, `group_membership`, `object_access`, `datasets_info` and `data_dict`.  
* Confirm the tables via the Athena console.

For instructions on building an Athena table with CloudTrail events, see [Amazon QuickSight Now Supports Audit Logging with AWS CloudTrail](https://aws.amazon.com/blogs/big-data/amazon-quicksight-now-supports-audit-logging-with-cloudtrail/)

* After all five tables are created in Athena, go to the security permissions on the QuickSight console to enable bucket access for s3://admin-console[AWS-account-ID] and s3://cloudtrail-awslogs-[aws-account-id]-do-not-delete (or the appropriate S3 bucket if you used a different name).
* Go to Lakeformation to grant the database and table access to QuickSight service role and the admin console execution role
* Enable Athena access under Security & Permissions.<br>
Now QuickSight can access all five tables through Athena.

### CloudFormation template for QuickSight objects

To create the QuickSight objects, complete the following steps:

* Get the QuickSight admin user’s ARN by running following command in the AWS Command Line Interface (AWS CLI):
`aws quicksight describe-user --aws-account-id [aws-account-id] --namespace default --user-name [admin-user-name]` 

For example: arn:aws:quicksight:us-east-1:12345678910:user/default/admin/xyz.  

In [26]:
import json
import boto3
import logging
import csv
import io
import os
import tempfile
from typing import Any, Callable, Dict, List, Optional, Union
import sys
import botocore

#start-Initial set up for the sdk env#
def default_botocore_config() -> botocore.config.Config:
    """Botocore configuration."""
    retries_config: Dict[str, Union[str, int]] = {
        "max_attempts": int(os.getenv("AWS_MAX_ATTEMPTS", "5")),
    }
    mode: Optional[str] = os.getenv("AWS_RETRY_MODE")
    if mode:
        retries_config["mode"] = mode
    return botocore.config.Config(
        retries=retries_config,
        connect_timeout=10,
        max_pool_connections=10,
        user_agent_extra=f"qs_sdk_admin_console",
    )
sts_client = boto3.client("sts", config=default_botocore_config())
account_id = sts_client.get_caller_identity()["Account"]
aws_region = 'us-east-1'
qs_client = boto3.client('quicksight', region_name=aws_region, config=default_botocore_config())


arn = qs_client.describe_user(
    UserName='admin/wangzyn-Isengard',
    AwsAccountId=account_id,
    Namespace='default'
)

arn = arn['User']['Arn']
arn

'arn:aws:quicksight:us-east-1:499080683179:user/default/admin/wangzyn-Isengard'

### Choose the below Launch Stack to create the QuickSight objects.

In [1]:
from IPython.display import HTML

image_url = "https://d32le6a78ncrct.cloudfront.net/image/Picture1.png"
link_url = "https://us-east-1.console.aws.amazon.com/s3/object/admin-console-cfn-dataprepare-code?region=us-east-1&bucketType=general&prefix=cfn/admin-console-qs-analysis-template-2025-added-datasets.template"

HTML(f'<a href="{link_url}" target="_blank"><img src="{image_url}" width="300"/></a>')

**Set the CFN execution role to be arn:aws:iam::[accountId]:role/QuickSightAdminConsole2025**